In [ ]:
!pip install transformers
!pip install textpruner
!nvidia-smi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 70.8 MB/s eta 0:00:00
  Created wheel for textpruner: filename=textpruner-1.1.post2-py3-none-any.whl size=43877 sha256=0c939f44c472c964718014248b2869e10477b20449cc0e004006495a2bc2b5e5
  Stored in directory: /root/.cache/pip/wheels/68/7c/60/742b2f0783fecc5b0c88d9f5adc53728ab1863d17398f842d2
Successfully built textpruner
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling nump

Wed Nov 15 03:09:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import json
from transformers import AutoTokenizer, BertModel
import tqdm
import torch.nn as nn
import math
import pdb

In [ ]:
from transformers import BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
max_sequence_length = tokenizer.model_max_length
print(f"Maximum Sequence Length for {model_name}: {max_sequence_length}")

Maximum Sequence Length for bert-base-uncased: 512


In [ ]:
# custom dataset class to load our data

class OurDataset(Dataset):
    def __init__(self, data_file, labels_file):
        self.full_data = json.load(open(data_file))
        self.labels = torch.load(labels_file)

        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.model = BertModel.from_pretrained("bert-base-uncased")

    def __len__(self):
        return 1000
        return len(self.full_data)

    def __getitem__(self, idx):
        #inputs = self.tokenizer(self.full_data[idx], return_tensors="pt")
        inputs = self.tokenizer(self.full_data[idx], return_tensors="pt", truncation=True, padding=True)
        outputs = self.model(**inputs)
        # last_hidden_states = outputs.last_hidden_state[0][0][:512]  # we want to Trim to size 512
        last_hidden_states = outputs.last_hidden_state[0][0]
        #return inputs, self.labels[idx]
        #return self.full_data[idx], self.labels[idx]
        # pdb.set_trace()
        return torch.unsqueeze(last_hidden_states,0).to('cuda:0'), self.labels[idx][0].long().to('cuda:0')
        #return self.tokenizer(self.full_data[idx], return_tensors="pt", truncation=True, padding=True), self.labels[idx]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

ds = OurDataset("full_data.json", "labels.torch")
dl = DataLoader(ds)
print(ds[1][0].shape)

# Assuming you have a DataLoader `data_loader` and the total dataset size `total_data_len`
# Example: DataLoader was created from a TensorDataset
# dataset = TensorDataset(X, y)
total_data_len = len(dl.dataset)

# Determine the sizes for the training and testing sets
train_size = int(0.7 * total_data_len)  # 70% for training
test_size = total_data_len - train_size  # Remaining 30% for testing

# Split the DataLoader into train and test DataLoaders
train_data, test_data = torch.utils.data.random_split(dl.dataset, [train_size, test_size])

# Create separate DataLoader objects for training and testing
train_loader = DataLoader(train_data, batch_size=dl.batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=dl.batch_size, shuffle=False)

# Now you have separate DataLoader objects for training and testing

torch.Size([1, 768])


In [ ]:
import torch.nn
from textpruner import TransformerPruner, TransformerPruningConfig
from transformers import BertForMaskedLM
from typing import Optional
class myBert(BertForMaskedLM):
    def __init__(self, config):
        super().__init__(config)
        self.classifcation = nn.Linear(768,2)
        self.loss = loss = nn.CrossEntropyLoss()

    def forward(self,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        input_ids: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,

        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,return_logits=False):
      out = super().forward(inputs_embeds=inputs_embeds, output_hidden_states=True)
      logits = self.classifcation(out['hidden_states'][-1]).squeeze(1)
      if return_logits:
        return logits
      return self.loss(logits, labels)


In [ ]:
from textpruner import TransformerPruner, TransformerPruningConfig
from transformers import BertForMaskedLM

model = myBert.from_pretrained('bert-base-uncased').to('cuda:0')

In [ ]:
from textpruner import TransformerPruner, TransformerPruningConfig
from transformers import BertForMaskedLM

model = myBert.from_pretrained('bert-base-uncased').to('cuda:0')
transformer_pruning_config = TransformerPruningConfig(
      target_ffn_size=2048,
      target_num_of_heads=8,
      pruning_method='iterative',
      n_iters=2)
pruner = TransformerPruner(model,transformer_pruning_config=transformer_pruning_config)
pruner.prune(dataloader=train_loader,save_model=True)

In [ ]:
import torch
a = torch.load('/content/pruned_models/pruned_H8.0F2048/pytorch_model.bin')

model.load_state_dict(a)
#print(a)

<All keys matched successfully>

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.type(torch.float32)
        y_pred = model(inputs_embeds = X_batch, return_logits=True)
        y_pred_class = y_pred.argmax(dim=1)  # get the predicted class
        correct += (y_pred_class == y_batch.sum().item())
        total += y_batch.size(0)

acc = correct / total
print("Model accuracy: %.2f%%" % (acc*100))

Model accuracy: 56.33%
